In [1]:
from datetime import datetime
from planet import Auth
from planet import Session, data_filter
import json
import time
import os

# We will also create a small helper function to print out JSON with proper indentation.
def indent(data):
    print(json.dumps(data, indent=2))

In [4]:
with open(os.path.join('geojson', "map.geojson")) as f:
    geom = json.loads(f.read())

In [19]:
import asyncio
import os

import planet

DOWNLOAD_DIR = os.getenv('TEST_DOWNLOAD_DIR', '.')

with open(os.path.join('geojson', "map.geojson")) as f:
    geom = json.loads(f.read())


def create_requests():
    # The Orders API will be asked to mask, or clip, results to
    # this area of interest.
    iowa_aoi = geom

    # In practice, you will use a Data API search to find items, but
    # for this example take them as given.
    iowa_items = list_id

    iowa_order = planet.order_request.build_request(
        name='Vodamulaa_Maldives_Jan_June_2019',
        products=[
            planet.order_request.product(item_ids=iowa_items,
                                         product_bundle='analytic_udm2',
                                         item_type='PSScene')
        ],
        tools=[planet.order_request.clip_tool(aoi=iowa_aoi)])

    return [iowa_order]


async def create_and_download(client, order_detail, directory):
    """Make an order, wait for completion, download files as a single task."""
    with planet.reporting.StateBar(state='creating') as reporter:
        order = await client.create_order(order_detail)
        reporter.update(state='created', order_id=order['id'])
        await client.wait(order['id'], callback=reporter.update_state)

    await client.download_order(order['id'], directory, progress_bar=True)


async def main():
    async with planet.Session() as sess:
        client = sess.client('orders')

        requests = create_requests()

        await asyncio.gather(*[
            create_and_download(client, request, DOWNLOAD_DIR)
            for request in requests
        ])


if __name__ == '__main__':
   await main()

00:00 - order  - state: creating


BadQuery: {"field":{"Details":[{"message":"No access to assets: PSScene/20190225_050452_0f52/[ortho_analytic_4b ortho_analytic_4b_xml]"},{"message":"No access to assets: PSScene/20190207_050731_0f4d/[ortho_analytic_4b ortho_analytic_4b_xml]"},{"message":"No access to assets: PSScene/20190126_050234_101e/[ortho_analytic_4b ortho_analytic_4b_xml]"}]},"general":[{"message":"Unable to accept order"}]}


In [14]:
from osgeo import gdal
import matplotlib.pyplot as plt
data = gdal.Open(os.path.join('c6d972cb-04dd-4b3f-bb29-7177593144d2', 'PSScene', '20231202_044119_76_24af_3B_AnalyticMS_clip.tif'))
blue = data.GetRasterBand(1).ReadAsArray()

c:\Users\mp222\.conda\envs\IslandTime\Lib\site-packages\matplotlib\colors.py:1099: RuntimeWarning: coroutine 'main' was never awaited
  data_r = {key: (functools.partial(self._reverser, data)


In [18]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if 'PL_API_KEY' in os.environ:
    API_KEY = os.environ['PL_API_KEY']
else:
    API_KEY = 'PLAK03522c42c1ee4651a5b0e983b599f1c7'
    os.environ['PL_API_KEY'] = API_KEY

client = Auth.from_key(API_KEY)

In [12]:
# Define the filters we'll use to find our data

item_types = ["PSScene"]

geom_filter = data_filter.geometry_filter(geom)
clear_percent_filter = data_filter.range_filter('clear_percent', 90)
date_range_filter = data_filter.date_range_filter("acquired", gt = datetime(month=1, day=1, year=2019), lt = datetime(month=6, day=30, year=2019))
cloud_cover_filter = data_filter.range_filter('cloud_cover', None, 0.3)

combined_filter = data_filter.and_filter([geom_filter, clear_percent_filter, date_range_filter, cloud_cover_filter])

In [13]:
async with Session() as sess:
    cl = sess.client('data')
    request = await cl.create_search(name='Vodamulaa_Maldives_2019', search_filter=combined_filter, item_types=item_types)

In [14]:
# Search the Data API

# The limit paramter allows us to limit the number of results from our search that are returned.
# The default limit is 100. Here, we're setting our result limit to 50.
async with Session() as sess:
    cl = sess.client('data')
    items = cl.run_search(search_id=request['id'], limit=1000)
    item_list = [i async for i in items]

In [15]:
list_id = [item_list[i]['id'] for i in range(len(item_list))]

In [16]:
len(list_id)

20

In [39]:
for item in item_list:
    print(item['id'], item['properties']['item_type'])

20191217_061008_03_1063 PSScene
20190903_061431_79_1068 PSScene
20190903_061433_89_1068 PSScene
20190331_062214_80_1062 PSScene
20190317_050409_1042 PSScene
20191228_051623_1105 PSScene
20191222_051004_103d PSScene
20191217_045841_0e19 PSScene
20191217_045840_0e19 PSScene
20191203_050851_1011 PSScene
20191203_042315_0f49 PSScene
20191127_051015_0f22 PSScene
20191124_050929_1001 PSScene
20191119_042609_1049 PSScene
20191118_042557_1_0f32 PSScene
20191103_050935_0f3f PSScene
20191102_052636_21_1057 PSScene
20191009_043119_0f2a PSScene
20191008_050813_0f25 PSScene
20191002_052447_21_105d PSScene
20190918_050749_103d PSScene
20190918_050748_103d PSScene
20190916_050914_1003 PSScene
20190910_051017_0f34 PSScene
20190908_050616_1003 PSScene
20190905_043654_104a PSScene
20190905_043655_104a PSScene
20190903_043713_0f2b PSScene
20190902_050848_103d PSScene
20190902_050849_103d PSScene
20190827_050835_1025 PSScene
20190820_043936_1049 PSScene
20190820_043935_1049 PSScene
20190814_050704_1010 PS

In [12]:
path_ouput = os.path.join(os.getcwd(), 'data', 'coastsat_data', 'Vodamulaa_Maldives', 'PSScene')
with open(os.path.join(path_ouput, 'results.json'),'w') as f:
    jsonStr = json.dumps(item_list)
    f.write(jsonStr)
    f.close()

In [15]:
# Define our item_id, and item_type
item = item_list[0]
item_id = item['id']
item_type = item['properties']['item_type']

In [18]:
# Analytic Asset
async with Session() as sess:
    cl = sess.client('data')
    # Get Asset
    asset_desc = await cl.get_asset(item_type_id=item_type,item_id=item_id, asset_type_id='ortho_analytic_4b')
    # Activate Asset
    await cl.activate_asset(asset=asset_desc)
    # Wait Asset
    await cl.wait_asset(asset=asset_desc)
    # Download Asset
    asset_path = await cl.download_asset(asset=asset_desc, directory=path_ouput, overwrite=True)

c:\Users\mp222\OneDrive - Imperial College London\IslandTimeGitHub\IslandTime\data\coastsat_data\Vodamulaa_Maldives\PSScene\20231023_052057_13_2483_3B_AnalyticMS.tif: 100%|██████████| 467k/467k [00:12<00:00, 38.9MB/s] 


In [ ]:
# Analytic XML Asset
async with Session() as sess:
    cl = sess.client('data')
    # Get Asset
    asset_desc = await cl.get_asset(item_type_id=item_type,item_id=item_id, asset_type_id='ortho_analytic_4b_xml')
    # Activate Asset
    await cl.activate_asset(asset=asset_desc)
    # Wait Asset (this may take some time!)
    await cl.wait_asset(asset=asset_desc)
    # Download Asset
    asset_path = await cl.download_asset(asset=asset_desc, directory=path_ouput, overwrite=True)